In [1]:
import os
import time

import itertools as it
import multiprocessing as mp

import numpy as np

from importlib import reload

locationPrefix = "/home/tfuser/programming"
homeLocation = os.getcwd()
libsLocation = os.path.join(locationPrefix, "gdrove", "tf")

os.chdir(libsLocation)
import tsp.tsp_proc as tsp
import all_aux as au
os.chdir(homeLocation)


print("Ok!")

Ok!


In [2]:
class TspMsg:
    def __repr__(self):
        return("Abstract TSP message")
    def __str__(self):
        return("Abstract TSP message")

class PoisonMsg(TspMsg):
    def __repr__(self):
        return("TSP Poison message")
    def __str__(self):
        return("TSP Poison message")

class AskLoadMsg(TspMsg):
    def __repr__(self):
        return("TSP Ask for load message")
    def __str__(self):
        return("TSP Ask for load message")

class LoadMsg(TspMsg):
    def __init__(self, load):
        self.load = load
    def __repr__(self):
        return("TSP Load message: {}".format(self.load))
    def __str__(self):
        return("TSP Load message: {}".format(self.load))
        
class InfoMsg(TspMsg):
    def __init__(self, info):
        self.info = info
    def __repr__(self):
        return("TSP Info message: {}".format(self.info))
    def __str__(self):
        return("TSP Info message: {}".format(self.info))
        
        
genMsg = TspMsg()
poiMsg = PoisonMsg()
loaMsg = LoadMsg([42, 21])

print(isinstance(genMsg, TspMsg))
print(isinstance(genMsg, PoisonMsg))
print(isinstance(poiMsg, PoisonMsg))
print(isinstance(poiMsg, TspMsg))
print(isinstance(loaMsg, TspMsg))
print(loaMsg)
print(poiMsg)

True
False
True
True
True
TSP Load message: [42, 21]
TSP Poison message


In [ ]:
def rouGenWorkerFn(queIn, queOut, queHost, name=None):
    def underhood(indices):
        gen = it.permutations(indices)
        
        running = True
        while running:
            msg = queIn.get()
            
            if isinstance(msg, PoisonMsg):
                running = False
                queHost.put(InfoMsg(info={"summary":"GEN: poisoned"}))
                
            elif isinstance(msg, AskLoadMsg):
                try:
                    route = next(gen)
                    queOut.put(LoadMsg(load=route))
                except StopIteration:
                    queHost.put(InfoMsg(info={"summary":"GEN: routes exghausted"}))
                    queHost.put(PoisonMsg())
                    
            else:
                queHost.put(InfoMsg(info={"summary": "GEN: received message of unknown purpose"}))
                
        ##
        return()
    
    ##
    if name is not None:
        underhood.__name__ = name
    return(underhood)




##  ##  TEST
queIn = mp.Queue()
queHost = mp.Queue()

worker = mp.Process(target=rouGenWorkerFn(queIn=queIn, queOut=queHost, queHost=queHost, name="WORKER_FN"),
                    args=([0,1,2,3],), name="WORKER")
worker.start()

queIn.put(AskLoadMsg())
msg = queHost.get()
try:
    print(msg.load)
except Exception as e:
    print("ERROR:", e)
    
    
queIn.put(AskLoadMsg())
msg = queHost.get()
try:
    print(msg.load)
except Exception as e:
    print("ERROR:", e)
    
    
queIn.put(PoisonMsg())
msgF = queHost.get()
try:
    print(msgF.info["purpose"])
except Exception as e:
    print("ERROR:", e)
    
## NEXT TEST INPUT
comb = msg.load    

worker.join()
print("Ok! Worker stopped: {}".format(not worker.is_alive()))

In [15]:
def xyDistMatrix(xs, ys):
    diffXs = np.stack([np.abs(x) - np.abs(xs) for x in xs])
    diffYs = np.stack([np.abs(y) - np.abs(ys) for y in ys])
    
    ##
    return(  np.sqrt(np.square(diffXs) + np.square(diffYs))  )



def xyCountRoute(distMatrix, name=None):
    def underhood(route):
        runs = zip(route[:-1], route[1:])
        return(  sum([distMatrix[run] for run in runs])  )
    
    ##
    if name is not None:
        underhood.__name__ = name
    return(underhood)



def countRouteFn(distMatrix, counterReportPeriod=1e6, name=None):
    def underhood(queIn, queOut, queHost):
        minLength = 999.0
        minRoute = None
        counter = 0
        
        ## TBD
        queHost.put(InfoMsg(info="CNT: started"))
        
        running = True
        while running:
            msg = queIn.get()
            
            if isinstance(msg, PoisonMsg):
                running = False
                
            elif isinstance(msg, LoadMsg):
                ## EXTRACT ROUTE 
                route = msg.load
                
                ## (e.g. (0,3,4,2,5,1,0) <- this one is reverse)
                route = (0,) + route + (0,)
                
                ## CHECK FOR REVERSE & COUNT
                if route[1] < route[-2]:
                    length = xyCountRoute(distMatrix)(route)
                    if length < minLength:
                        minLength = length
                        minRoute = route

                    counter += 1                    
                    if (counterReportPeriod % counter == 0) and (counter > 0):
                        queHost.put(InfoMsg(info={"summary": "CNT: Routes count: {}".format(counter),
                                                  "count": counter}))
                        counter = 0
                
                ## READY FOR NEW ROUTE
                queOut.put(AskLoadMsg())                        
                    
            else:
                queHost.put(InfoMsg(info="CNT: received message of unknown purpose"))
        
        ##
        queHost.put(InfoMsg(info="CNT: stopping and sending the shortest route found"))
        queHost.put(InfoMsg(info={"summary": "CNT: Shortest route found: {} ==> {}".format(minRoute, minLength),
                                  "minLength": minLength,
                                  "minRoute": minRoute}))
        
        ##
        return()
    
    ##
    if name is not None:
        underhood.__name__ = name
    return(underhood)




##  ##  TESTS
if False:
    ##  ## TEST 1: xyDistMatrix
    townIds = [0,1,2,3,4,5]
    xs = np.random.uniform(size=(len(townIds),), low=-1.0, high=1.0)
    ys = np.random.uniform(size=(len(townIds),), low=-1.0, high=1.0)
    distMatrix = xyDistMatrix(xs, ys)
    print(distMatrix)
    
    
if False:
    print()
    ##  ## TEST 2: xyCountRoute
    townIds = [0,1,2,3,4,5]
    xs = np.random.uniform(size=(len(townIds),), low=-1.0, high=1.0)
    ys = np.random.uniform(size=(len(townIds),), low=-1.0, high=1.0)
    distMatrix = xyDistMatrix(xs, ys)
    print(distMatrix)
    length012345 = xyCountRoute(distMatrix)(tuple(townIds))
    print(length012345)
    

#if True:
##  ## TEST 3: countRouteFn
townIds = list(range(11))
xs = np.random.uniform(size=(len(townIds),), low=-1.0, high=1.0)
ys = np.random.uniform(size=(len(townIds),), low=-1.0, high=1.0)
distMatrix = xyDistMatrix(xs, ys)

gen = it.permutations(townIds[1:]) ## except start point

##  ##
queToCnt = mp.Queue()
queToGen = mp.Queue()
queToHost = mp.Queue()

routeCounter = mp.Process(target=countRouteFn(distMatrix=distMatrix, counterReportPeriod=50000,
                                              name="Route_Counter_Fn"),
                          args=(queToCnt, queToGen, queToHost), name="Route_Counter")

timeStart = time.time()
routeCounter.start()

##
running = True
while running:
    try:
        comb = next(gen)
        queToCnt.put(LoadMsg(load=comb))
        askForLoad = queToGen.get()
    except StopIteration:
        running = False

##
queToCnt.put(PoisonMsg())


routeCounter.join()
while not queToHost.empty():
    msg = queToHost.get()
    print(msg)

queToCnt.close()
queToGen.close()
queToHost.close()

timeTaken = time.time() - timeStart
print("Ok! Worker stopped: {}. Time taken: {} sec.".format(not routeCounter.is_alive(), "%.2f"%timeTaken))

TSP Info message: CNT: started
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Info message: {'summary': 'CNT: Routes count: 50000', 'count': 50000}
TSP Inf

In [ ]:
## EMERGENCY EXTRACTION
while not queToHost.empty():
    msg = queToHost.get()
    print(msg)
    
queToCnt.close()
queToGen.close()
queToHost.close()
    
print("Ok!")

In [16]:
##  ##  TEST without GEN

townIds = list(range(11))
numCounterWorkers = 44

##  ##

gen = it.permutations(townIds[1:])

xs = np.random.uniform(size=(len(townIds),), low=-1.0, high=1.0)
ys = np.random.uniform(size=(len(townIds),), low=-1.0, high=1.0)

distMatrix = xyDistMatrix(xs, ys)

##  ##

queToGen = mp.Queue()
queToCnt = mp.Queue()
queToHost = mp.Queue()

cntWorkers = [mp.Process(target=countRouteFn(distMatrix=distMatrix, counterReportPeriod=int(1e5),
                                             name="Route_Counter_Fn_{}".format(no)),
                         args=(queToCnt, queToHost, queToHost),
                         name="Route_Counter_{}".format(no)) \
              for no in range(numCounterWorkers)]

for worker in cntWorkers:
    worker.start()
    
print("HOST: All workers started: {}".format(all([worker.is_alive() for worker in cntWorkers])))

timeStart = time.time()

candidates = []
counted = 0
running = True

totalRoutesCount = tsp.phuc(len(townIds) - 1) // 2
for _ in range(100):
    queToHost.put(AskLoadMsg())

while running:
    msg = queToHost.get()
    
    if isinstance(msg, PoisonMsg):
        running = False
        
    elif isinstance(msg, AskLoadMsg):
        try:
            route = next(gen)
            queToCnt.put(LoadMsg(load=route))
        except StopIteration:
            running = False
        
    elif isinstance(msg, InfoMsg):
        info = msg.info
        
        if isinstance(info, dict):
            if info["summary"].startswith("CNT: Routes count"):
                counted += info["count"]
                print("HOST: Count routes {} / {}".format(counted, totalRoutesCount))
                
            elif info["summary"].startswith("CNT: Shortest route found"):
                candidates += [{"minRoute": info["minRoute"],
                                "minLenght": info["minLength"]}]
                
            else:
                pass
            
        else:
            print(info)        
    
    else:
        print("HOST: Recieved message of unknown purpose")
        
print("HOST: shutting down")

for _ in range(numCounterWorkers):
    queToCnt.put(PoisonMsg())

while not queToHost.empty():
    msg = queToHost.get()
    
    if isinstance(msg, InfoMsg):
        info = msg.info
        
        if isinstance(info, dict):
            if info["summary"].startswith("CNT: Routes count"):
                counted += info["count"]
                print("HOST: Count routes {} / {}".format(counted, totalRoutesCount))
                
            elif info["summary"].startswith("CNT: Shortest route found"):
                candidates += [{"minRoute": info["minRoute"],
                                "minLength": info["minLength"]}]
                
            else:
                pass
            
        else:
            print(info)  
    
    else:
        pass

## CANDIDATES
minRoute = None
minLength = 999.0
for c in candidates:
    if c["minLength"] < minLength:
        minLength = c["minLength"]
        minRoute = c["minRoute"]
print("\nShortest route: {} with Length: {}\n".format(minRoute, "%.5f"%minLength))

for worker in cntWorkers:
    worker.join()
    
queToGen.close()
queToCnt.close()
queToHost.close()
    
print("All workers stopped: {}".format(all([not worker.is_alive() for worker in cntWorkers])))
timeTaken = time.time() - timeStart
print("Ok! Time taken: {} sec.".format("%.2f"%timeTaken))

HOST: All workers started: True
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
CNT: started
HOST: shutting down
CNT: stopping and sending the shortest route found
CNT: stopping and sending the shortest route found
CNT: stopping and sending the shortest route found
CNT: stopping and sending the shortest route found
CNT: stopping and sending the shortest route found
CNT: stopping and sending the shortest route found
CNT: stopping and sending the shortest route found
CNT: stopping and s

In [ ]:
## EMERGENCY SHUT DOWN
for worker in cntWorkers:
    queToCnt.put(PoisonMsg)
    
for worker in cntWorkers:
    worker.join()
    
for que in [queToGen, queToCnt, queToHost]:
    que.close()
    
print("Ok!")

In [17]:
'''
1 GEN vs. CNTs:
1:1 ==> 570.17 sec.
1:22 ==> 237.48 sec.
1:44 ==> 241.91 sec.
'''

print("Only reports valued")

Only reports valued
